# MNIST from scratch

In this notebook, I am implementing a model for solving the MNIST competition from scratch. [Chapter 4](https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb) of the book is my guide, and I will implement this in analogy.

I skip over the exercise to create a model based on the average pixel values of each digit, instead, I jump straight to the linear model / neural net. I added text where I felt that it improved my understanding of what is going on, or where I felt that the book was too compact. In any case I recommend also reading along in the book, as I do not intend to re-write the chapter in this notebook. For easy reference, I kept the main headings the same as in the book.

## Converting the Data

The first task is to convert the `csv`-files into tensors so that we can work with it.

Let's load the data:

In [1]:
from fastai.vision.all import *
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For the initial phases of writing this notebook, I limited the number of images to a smaller size, so that things were easier visualize, easier to analyze etc. As it turned out, this was useful for the data conversion and for creating the loss function, but afterwards I removed the limit. For historic reason, I leave it in the notebook:

In [3]:
limit = 0

In [4]:
if limit > 0:
    train = train[:limit]

len(train)

42000

Let's split the data into a training set and a validation set. Just to state the obvious: We cannot use the test set as a validation set because it is not labeled.

In [5]:
train_percentage = 0.8
split_index = int(len(train) * train_percentage)
split_index
valid = train[split_index:]
train = train[:split_index]
len(train), len(valid)

(33600, 8400)

The independent variables `x` are the images (i.e. the pixels of each 28x28 image) and the dependent variable `y`are the lables.

The images reside in the second to the last column of the training dataframe.

The labels reside in the first column of the training dataframe.

For converting the data, the following magic is contained in the next line of code:

* `pd.DataFrame.values`, i.e. the `.values` "returns a Numpy representation of the DataFrame" (see `pd.DataFrame.values?`)
* to normalize the pixel values to values between 0 and 1 I divided them by 255 (while writing this up, I am actually not sure if that is needed, but somehow it feels like common practice to do it...)
* to avoid type conflicts all elements are converted to floats by adding `.float()`

In [6]:
train_x = torch.tensor((train.iloc[:,1:].values)/255).float()
train_x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [7]:
train_x.size(), train_x.shape

(torch.Size([33600, 784]), torch.Size([33600, 784]))

Doing the same for the variables, the dependent variable:

In [8]:
train_y = torch.tensor(train.iloc[:,0].values).float()
train_y

tensor([1., 0., 1.,  ..., 0., 2., 2.])

In [9]:
train_y.size(), train_y.shape

(torch.Size([33600]), torch.Size([33600]))

This  looks slightly different in the book... It turns out, we need to convert this 1D tensor into a 2D tensor so that we can use it in our calculations.

The line of code below is inspired by <https://discuss.pytorch.org/t/convert-1d-tensor-to-2d-tensor/5599/4>

In [10]:
train_y = train_y.view(len(train_y), 1)
train_y[:5]

tensor([[1.],
        [0.],
        [1.],
        [4.],
        [0.]])

In [11]:
train_y.size(), train_y.shape

(torch.Size([33600, 1]), torch.Size([33600, 1]))

Now the result looks like in the book :).

Finally, let's create a dataset, i.e. a collection of tuples `(x,y)`

In [12]:
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape,y

(torch.Size([784]), tensor([1.]))

Let's repeat for the validation set:

In [13]:
valid_x = torch.tensor((valid.iloc[:,1:].values/255)).float()
valid_y = torch.tensor(valid.iloc[:,0].values).float()
valid_y = valid_y.view(len(valid_y), 1)
valid_dset = list(zip(valid_x,valid_y))
valid_x.shape, valid_y.shape

(torch.Size([8400, 784]), torch.Size([8400, 1]))

## The Loss Function

### Initializing the parameters

To get started working on the loss function, we need to initialize the parameters:

In [14]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [15]:
weights = init_params((28*28,1))
weights[:5]

tensor([[-0.9063],
        [-0.6531],
        [-0.0431],
        [-3.2855],
        [ 0.4129]], grad_fn=<SliceBackward0>)

In [16]:
weights.size()

torch.Size([784, 1])

In [17]:
bias = init_params(1)
bias

tensor([-0.2507], requires_grad=True)

### Short excursion on matrix multiplication

Calculating the prediction for the first element:

> Note: `weights.T` is the transposed tensor. For doing this "normal" multiplication the tensor of the individual image needs to be "flipped". When using the matrix multiplication, the regular version of our tensor is used. As you can see the 2 following cells return the same result:

In [18]:
(train_x[0]*weights.T).sum() + bias # T is the transposed tensor

tensor([-5.5514], grad_fn=<AddBackward0>)

In [19]:
(train_x[0]@weights).sum() + bias

tensor([-5.5514], grad_fn=<AddBackward0>)

Revisiting matrix multiplication: As described on more detail in my blog post on [matrix multiplication](https://chrwittm.github.io/posts/2022-10-28-matrix-multiplication/), the number of columns of matrix $A$ need to match the number of lines of matrix $B$:

![Matrix dimensions](https://chrwittm.github.io/posts/2022-10-28-matrix-multiplication/khan-matrix-dimensions.png)

_Illustration [by Khan Academy CC BY-NC-SA 3.0 US](https://www.khanacademy.org/)_
_[Note](https://support.khanacademy.org/hc/en-us/articles/202262954-Can-I-use-Khan-Academy-s-videos-name-materials-links-in-my-project-): All Khan Academy content is available for free at (www.khanacademy.org)_

We can nicely see this when looking at the shape of our tensors (note that the values `784` in the output of the second line are next to each other):

In [20]:
print(f'This does not work: {train_x[0].shape}, {weights.T.shape}')
print(f'This works: {train_x[0].shape}, {weights.shape}')

This does not work: torch.Size([784]), torch.Size([1, 784])
This works: torch.Size([784]), torch.Size([784, 1])


Same for the whole matrix:

In [21]:
print(f'This does not work: {train_x.shape}, {weights.T.shape}')
print(f'This works: {train_x.shape}, {weights.shape}')

This does not work: torch.Size([33600, 784]), torch.Size([1, 784])
This works: torch.Size([33600, 784]), torch.Size([784, 1])


In [22]:
train_x.shape, weights.T.shape

(torch.Size([33600, 784]), torch.Size([1, 784]))

### Calculating the predictions (forward pass)

In our current simple model, we matrix multiply the parameters with the weights (+ bias).

Let's wrap that in a function, as suggested in the book. 

> Personal Note 1: I do not really like the global usage of the weight and bias in the function...

> Personal Note 2: Is it actually important to wrap this in a function because otherwise it cannot calculate the gradients?

Let's Calculate the predictions for the whole training data:

In [23]:
def linear1(xb): return xb@weights + bias

In [24]:
preds1 = linear1(train_x)
preds1[:5]

tensor([[-5.5514],
        [-1.4782],
        [10.9867],
        [ 0.3375],
        [-0.5475]], grad_fn=<SliceBackward0>)

Notice: The first element of the tensor is the same as the value we calculated above.

Let's compare the predictions with the actual labels:

In [25]:
preds1.round()[:5]

tensor([[-6.],
        [-1.],
        [11.],
        [ 0.],
        [-1.]], grad_fn=<SliceBackward0>)

In [26]:
train_y[:5]

tensor([[1.],
        [0.],
        [1.],
        [4.],
        [0.]])

In [27]:
corrects = preds1.round() == train_y
corrects[:5] # all wrong

tensor([[False],
        [False],
        [False],
        [False],
        [False]])

In [28]:
corrects.float().mean()

tensor(0.0317)

As it turns out, the vast majority of the predictions is wrong. This is not really surprising, because we randomly initialized the parameters.

> Note: Normalizing the pixel values to values between 0 and 1 sure helped this result. Without the normalization, we would have zero corrects.

Actually, I was struggling with the fact that there are any correct values because essentially, everything is this calculation (so far) is random. I was also struggling with the fact that you could simply define which results of the net represent correct values. (At this point in the book, the define that a `0` means a `3` and a `1` means a `7`.) And essentially, you can make up anything as long as it is consistent, because the model will learn these values because that is what gradient descent will descent to.

### Defining the loss function

Now we can define the loss function: Let's use the mean squared error between the predictions and the labels:

In [29]:
def mnist_loss(preds, targets):
    return ((preds-targets)**2).mean()

In [30]:
loss1 = mnist_loss(preds1, train_y)
loss1

tensor(147.2167, grad_fn=<MeanBackward0>)

### Manual backpropagation

After having calculated the loss, we can do a bit of backpropagation to see if we are on the right track.

Calling `.backward()` will calculate the gradients, i.e. the effect that each individual pixel value has on the loss function, i.e. how the loss changes it we adjust the value of one pixel.

In [31]:
loss1.backward()

Adjusting the weights with a learning rate:

In [32]:
#weights = weights.grad * 0.001 #not sure about this one, I think this is an error, because I do not adjust the weight, but replace them

with torch.no_grad(): weights -= weights.grad*0.01 # this is better: update parameters

Doing the forward pass again:

In [33]:
preds2 = (train_x@weights + bias)
preds2[:5]

tensor([[-2.0176],
        [ 6.6123],
        [14.0653],
        [ 2.9795],
        [ 7.9765]], grad_fn=<SliceBackward0>)

In [34]:
loss2 = mnist_loss(preds2, train_y)
loss2

tensor(84.9808, grad_fn=<MeanBackward0>)

The loss went down :)

In [35]:
corrects = preds2.round() == train_y
corrects.float().mean()

tensor(0.0443)

Additionally, we have more corrects, so we did one optimization step, and it looks like it actually worked :).

But as described in the book, and also in some previous notebooks we also have to care about initializing gradients, so this would get pretty tedious.

Additionally, you would usually not use all the parameters for a training run, but instead, you would use mini-batches for training - which the dataloaders will do for us.

## Putting it all together

The following code, taken from the book, is quite compact, but actually pretty straight forward.

The following function, calculates the forward pass by calling `model(xb)`: In our context this actually `linear1(train_x)` because we pass `linear1` as the `model` and `train_x` as `xb`.

Afterward id calculates the loss and does the backpropagation.

In [36]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

For training a whole epoch (i.e. running over all the training data once), the following function iterates over the dataset stored in the dataloaders, calculated the gradients, updates the parameters and initializes the gradients.

In [37]:
def train_epoch(model, lr, params):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

The following function is my own implementation of determining the batch accuracy: A digit is correctly predicted if the rounding the predicted values reveals the digit.

In [38]:
def batch_accuracy(xb, yb):
    preds = xb.round()
    correct = preds == yb
    return correct.float().mean()

To validate an epoch, i.e. after having performed the training, i.e. the updates of the parameters, the following function iterates over the validation dataloaders, runs the forward pass (`model(xb)`) and compares that to the validation labels `yb`. The accuracy is returned as the`round`ed `mean()` of the individual tests.

In [39]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

### Reinitialize parameters

Let's start over again with some fresh set of parameters:

In [40]:
weights = init_params((28*28,1))
bias = init_params(1)

Using a dataloaders for managing the training data:

In [41]:
dl = DataLoader(dset, batch_size=256)
xb,yb = first(dl)
xb.shape,yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

The same for the validation set:

In [42]:
valid_dl = DataLoader(valid_dset, batch_size=256)

Without any training, we can still validate our parameters, which show some random corrects:

In [43]:
params = weights,bias
validate_epoch(linear1)

0.0449

Now, let's train one epoch:

In [44]:
lr = 0.005
params = weights,bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

0.097

That is a good improvement :)

> Note: One big learning here was that setting the training rate is SUPER important. The book uses a learning rate of `1`, but this would result in disaster for this model. We would always have zero corrects.

Let's do 20 more epochs:

In [45]:
for i in range(20):
    train_epoch(linear1, lr, params)
    print(validate_epoch(linear1), end=' ')

0.1103 0.1209 0.1289 0.1332 0.1354 0.1352 0.1383 0.1433 0.1437 0.1485 0.153 0.1541 0.1585 0.1596 0.1614 0.1627 0.1644 0.1648 0.1668 0.1692 

The results are not yet particularly great, but we can see some improvement.

## Creating an Optimizer

Let's re-implement what we did before with a Pytorch linear model:

In [46]:
linear_model = nn.Linear(28*28,1)

A new set of weights and biases:

In [47]:
w,b = linear_model.parameters()
w.shape,b.shape

(torch.Size([1, 784]), torch.Size([1]))

The following is taken from the book (again), and we can see that it essentially contains the same code as above.

In [48]:
class BasicOptim:
    def __init__(self,params,lr): self.params,self.lr = list(params),lr

    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr

    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None

In [49]:
opt = BasicOptim(linear_model.parameters(), lr)

In [50]:
def train_epoch(model):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

This is our starting point:

In [51]:
validate_epoch(linear_model)

0.1003

Let's train for 20 epochs:

In [52]:
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoch(model), end=' ')

In [53]:
train_model(linear_model, 20)

0.1769 0.193 0.202 0.205 0.2077 0.2074 0.2094 0.2121 0.2127 0.2138 0.2134 0.2139 0.2139 0.2148 0.215 0.215 0.2145 0.2151 0.2155 0.2165 

To be honest I am not quite sure, why this is performing a bit better...

Re-doing the same thing with a little bit more of Fast.AI magic:

In [54]:
linear_model = nn.Linear(28*28,1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)

0.1774 0.1933 0.2039 0.2065 0.2095 0.2085 0.2107 0.2116 0.2124 0.2121 0.2133 0.2145 0.2141 0.2167 0.2162 0.2162 0.217 0.2158 0.215 0.216 

And now we are ready to use a learner:

In [55]:
dls = DataLoaders(dl, valid_dl)

In [56]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [57]:
learn.fit(10, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,4.848422,4.484610,0.172024,00:00
1,4.280970,4.180988,0.193810,00:00
2,4.057806,4.017156,0.201429,00:00
3,3.920729,3.901211,0.204881,00:00
4,3.820063,3.811138,0.207738,00:00
5,3.740935,3.738425,0.209405,00:00
6,3.676762,3.678550,0.211071,00:00
7,3.623770,3.628623,0.211429,00:00
8,3.579461,3.586605,0.212024,00:00
9,3.542054,3.550979,0.214286,00:00


## Adding a Non-linearity

Let's create a basic neural net:

In [58]:
def simple_net(xb): 
    res = xb@w1 + b1
    res = res.max(tensor(0.0))
    res = res@w2 + b2
    return res

In [59]:
w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,1))
b2 = init_params(1)

As it turns out, we can define the same also like this:

In [60]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

Let's train it:

In [61]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [62]:
learn.fit(40, 0.01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,4.307751,3.880059,0.195833,00:00
1,3.544580,3.287405,0.246786,00:00
2,3.084344,2.799699,0.289167,00:00
3,2.647977,2.399291,0.329286,00:00
4,2.318112,2.133533,0.360476,00:00
5,2.069795,1.933588,0.383333,00:00
6,1.914878,1.806098,0.393810,00:00
7,1.787905,1.700211,0.407976,00:00
8,1.692105,1.620444,0.417500,00:00
9,1.597911,1.551217,0.423571,00:00


As it turns out, it performs a lot better than our linear model.

## Going Deeper

I wanted to see how adjusting the layout of the network can change the network performance, both in runtime and it accuracy.

Here is a network with 3 layers. Notice that you have to make sure that the number of output parameters of a layer need to match the number of input parameters of the next layer: 

In [63]:
simple_net2 = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [64]:
learn = Learner(dls, simple_net2, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [65]:
learn.fit(40, 0.01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,4.297549,3.415897,0.213333,00:00
1,3.281935,2.690592,0.302619,00:00
2,2.513027,2.102891,0.385357,00:00
3,1.984050,1.785641,0.407976,00:00
4,1.703177,1.586349,0.441310,00:00
5,1.549266,1.455132,0.477024,00:00
6,1.435295,1.367049,0.502024,00:00
7,1.338028,1.290225,0.527738,00:00
8,1.258634,1.235726,0.538095,00:00
9,1.187515,1.208292,0.536310,00:00


Again, we have a significant improvement :).

Let's add one more layer:

In [66]:
simple_net3 = nn.Sequential(
    nn.Linear(28*28,150),
    nn.ReLU(),
    nn.Linear(150,150),
    nn.ReLU(),
    nn.Linear(150,130),
    nn.ReLU(),
    nn.Linear(130,1)
)

In [67]:
learn = Learner(dls, simple_net3, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [68]:
learn.fit(40, 0.01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,4.431557,3.259079,0.206429,00:00
1,2.937935,2.220023,0.325952,00:01
2,2.019956,1.611896,0.471905,00:00
3,1.544070,1.360150,0.527262,00:00
4,1.243392,1.301756,0.541429,00:00
5,1.087905,1.027154,0.605595,00:00
6,0.959080,0.913137,0.629762,00:00
7,0.866579,0.862369,0.635952,00:00
8,0.796565,0.824086,0.648810,00:00
9,0.717660,0.798681,0.648452,00:00


Another significant improvement, but we start to see that the accuracy does not constantly go up, but it also goes down. I suspect that this can be fixed by adjusting the learning rate.

Let's add one more layer:

In [69]:
simple_net4 = nn.Sequential(
    nn.Linear(28*28,150),
    nn.ReLU(),
    nn.Linear(150,150),
    nn.ReLU(),
    nn.Linear(150,150),
    nn.ReLU(),
    nn.Linear(150,130),
    nn.ReLU(),
    nn.Linear(130,1)
)

In [70]:
learn = Learner(dls, simple_net4, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [71]:
learn.fit(40, 0.01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,4.584154,3.168080,0.208214,00:02
1,2.669653,2.105103,0.346071,00:01
2,1.799561,1.497062,0.427262,00:01
3,1.362472,1.169315,0.570595,00:01
4,1.116101,0.982570,0.616905,00:01
5,0.942988,0.886795,0.653929,00:01
6,0.831714,0.824128,0.669048,00:01
7,0.736036,0.770736,0.690595,00:01
8,0.663777,0.742197,0.695476,00:01
9,0.618736,0.737393,0.681667,00:01


Again, a good improvement, but the results oscillate more between epochs, something to look into. But for now, let's switch the perspective to complete the whole game, let's move on to doing the predictions of the test data.

Before we do that, something odd I was not able to solve: Somehow I could not export the model: Any idea why that is the case of how to fix it?

In [72]:
path = Path()
path

Path('.')

In [73]:
learn.export?

Signature:
learn.export(
    fname='export.pkl',
    pickle_module=<module 'pickle' from '/home/chrwittm/mambaforge/lib/python3.10/pickle.py'>,
    pickle_protocol=2,
)
Docstring: Export the content of `self` without the items and the optimizer state for inference
File:      ~/mambaforge/lib/python3.10/site-packages/fastai/learner.py
Type:      method


In [74]:
#learn.export()

If executed, the above cell results in

```
AttributeError: 'list' object has no attribute 'new_empty'
```

Any ideas how to fix this?

## Predictions of the test data

Let's use our trained model to create a submission for the kaggle competition - knowing that the result could easily be improved, but this is about playing the whole game.

### Loading test data

First, we need to load the test data:

In [75]:
test = pd.read_csv("test.csv")
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Failure mode: Cannot run `learn.predict`

Again, an unsolved mystery: I could not work out how to call `learn.predict` to get the predictions, also other attempts failed.

Let me document my attempts before moving on to a method which worked.

Just to make sure that we know what we want to predict, here is a digit:

In [76]:
def pil2fast2(pil):
    return PILImage.create(np.array(pil.convert('RGB')))

def get_fast_test_image(index):
    im = torch.tensor(test.iloc[index])
    pil = Image.frombytes('L', (28, 28), bytes(im))#.convert('RGB')
    fast = pil2fast2(pil)
    return fast

In [77]:
fast_digit = get_fast_test_image(0)
fast_digit

Getting the same as a tensor:

In [78]:
def get_test_image_as_tensor(index):
    digit = torch.tensor((test.iloc[index])/255).float()
    digit = digit.view(len(digit), 1)
    return digit

In [79]:
test_digit = get_test_image_as_tensor(0)
test_digit.size()

torch.Size([784, 1])

In [80]:
#learn.predict(test_digit.T)

The above cell fails with

```
AttributeError: 'list' object has no attribute 'decode_batch'
```

Any idea why?

Alternative approach which a new test data loaders:

In [81]:
test_x = torch.tensor((test.iloc[:,:].values)/255).float()
test_x.size()

torch.Size([28000, 784])

In [82]:
test_dl = learn.dls.test_dl(test_x)

In [83]:
#learn.get_preds(dl=test_dl)

The above fails with

```
TypeError: Sequential.forward() takes 2 positional arguments but 256 were given
```

Any ideas why?

Note: calling `learn.get_preds()` returns the predictions of the validation set, so how to I get it to run on the test data?

In [84]:
preds,targs = learn.get_preds()

In [85]:
len(preds)

8400

In [86]:
preds

tensor([[0.0621],
        [7.2984],
        [6.9915],
        ...,
        [7.1430],
        [6.1489],
        [8.5495]])

### Solution: Just call the forward pass

After trying the above I took a break, and while driving to the store for grocery shopping it suddenly occurred to me that it should be quite simple to make the predictions without the learner, you just need to call the forward pass of the trained model. And indeed, there is a `.forward()`-method :)

In [87]:
simple_net4.forward(test_digit.T)

tensor([[1.8600]], grad_fn=<AddmmBackward0>)

Indeed, our test digit is a 2:

In [88]:
fast_digit

So let's run that on the whole dataset by simply passing the whole data, rounding and converting to integer to get the right format for submitting to kaggle:

In [89]:
test_preds = simple_net4.forward(test_x).round().int()

In [90]:
len(test_preds)

28000

In [91]:
test_preds.data

tensor([[2],
        [0],
        [9],
        ...,
        [3],
        [9],
        [2]], dtype=torch.int32)

As a final step, let's create the `csv`-file:

In [92]:
submission = pd.DataFrame(columns=['ImageId', 'Label'])
submission['ImageId'] = range(1,len(test_preds)+1)
submission['Label'] = test_preds.squeeze().tolist()

In [93]:
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,3
4,5,3


Downloading the `csv`-file: (commented not to have a new version each time I run the notebook)

In [94]:
#submission.to_csv('submission_simple_net4.csv', index=False)

The submission turned out as expected with a similar accuracy as indicated by our validation set.

Not a perfect submission result, but great learnings along the way :)

![My third submission result](sub3-084817.png)